In [43]:
pip install galois

Note: you may need to restart the kernel to use updated packages.


In [44]:
import galois
import numpy as np
import threading
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import ThreadPoolExecutor

np.set_printoptions(threshold=np.inf)

In [45]:
#definisco un campo di Galois come nell'esempio 
field = galois.GF(127)

In [46]:
#definisco una matrice k x (n-k) dove n-k = 4
k = 126 #le righe
n_k = 126 #le colonne

#N.B: Less usa q = dimensione campo: 127; k = 126; n = 252 

In [47]:
#provo co due matrici diagonali NON funziona
valori_diagonale = np.arange(1,127)
A = np.diag(valori_diagonale)

perm = np.random.permutation(126) #permutazione degli indici casuale
A_primo = A[perm, :]
A_primo = A_primo[:, perm]

A = field(A)
A_primo = field(A_primo)

In [48]:
A = field.Random((k, n_k))
#print(A)

In [49]:
id_mat = np.eye(126, dtype=int)

cols_to_permute = np.random.choice(id_mat.shape[1], size=50, replace=False)
permuted_cols = np.random.permutation(cols_to_permute)
matrice_bin1 = id_mat.copy()
matrice_bin1[:, cols_to_permute] = id_mat[:, permuted_cols]
matrice_bin1 = field(matrice_bin1)

cols_to_permute = np.random.choice(id_mat.shape[1], size=50, replace=False)
permuted_cols = np.random.permutation(cols_to_permute)
matrice_bin2 = id_mat.copy()
matrice_bin2[:, cols_to_permute] = id_mat[:, permuted_cols]
matrice_bin2 = field(matrice_bin2)

print(matrice_bin1)
print(matrice_bin2)

[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [50]:
#np.dot prodotto di due  array
A_primo_intermedio = field(np.dot(matrice_bin1, A))
#print(A_primo_intermedio)

In [51]:
A_primo = matrice_bin1 @ A @matrice_bin2
A_primo = field(A_primo)
#print(A_primo)

In [52]:
#qui in accordo con slide 18 articolo devo ordinare la matrice A e A_primo per righe e colonne, ordinamento per riga 
#è dato dalla somma delle righe in ordine decrescente, mentre per colonna in ordine numerico del primo elemento e in caso di 
#parità del secondo e così via

#lista per conservare somme delle righe, inizializzata con gli zeri
somme_rig = [field(0)] * A_primo.shape[0]

#lista per memorizzare i primi elementi della colonna
primi_colonne = [0] * A_primo.shape[1]
#lista per memorizzare i secondi elemnti della colonna
secondi_colonne = [0] * A_primo.shape[1]
#lista per memorizzare i terzi elementi della colonna
terzi_colonne = [0] * A_primo.shape[1]
#lista per memorizzare i quarti elemnti della colonna
quarti_colonne = [0] * A_primo.shape[1]

#QUI SI POTREBBE RAGGIONARE SU QUANTI ELEMENTI SALVARE DI OGNI COLONNA POICHÈ LA POSSIBILITÀ CHE QUESTI SIANO UGUALI 
#PER PIÙ DI TOT RIGHE DIMINUISCE (1/PROB_CONFLITTO_RIGA1 * 1/PROB_CONFLITTO_RIGA2 ....) È DA VEDERE COME VENGONO CREATE 

#funzione per somma elementi nella colonna
def somma_riga(matrice, indice_riga):
    riga = matrice[indice_riga, :]
    somma = field(np.sum([riga])%127)
    return somma
    #print("Riga : ", riga) #questi vanno eliminati poi
    #print("Tipo riga: ", type(riga)) #questo è per far vedere che pur essendo di tipo GF il np.sum non funziona se non metto %8
    #print("Somma: ", somma) #questi vanno eliminati poi
    #print("Tipo somma: ",type(somma)) #questo è per far vedere che pur essendo di tipo GF il np.sum non funziona se non metto %8
    #print(somme_rig)

#vado ad estrarre il primo ed il secondo elemento di ogni colonna
def estrai_da_colonna(matrice, indice_colonna):
    colonna  = matrice[:, indice_colonna]
    return colonna[:4]  # Restituisce i primi quattro elementi della colonna



#eseguo in parallelo somma_riga
def esegui_somma_righe_parallelo(matrice):
    somme_rig = [0] * matrice.shape[0]  #inizializza lista per le somme

    with ThreadPoolExecutor() as executor:
        executor.map(somma_riga, [matrice]*matrice.shape[0], range(matrice.shape[0]))
    return somme_rig

#eseguo in parallelo estrazione
def esegui_estrazione_seriale(matrice):
    for indice_colonna in range(matrice.shape[1]):
        estrai_da_colonna(matrice, indice_colonna)

# Funzione per applicare una permutazione ciclica a un insieme di colonne
def permutazione_ciclica(matrice, blocco_dim):
    matrice_permutata = matrice.copy()
    
    # Eseguiamo una permutazione ciclica su tutte le colonne a gruppi di 'blocco_dim'
    for i in range(0, matrice.shape[1], blocco_dim):
        fine_blocco = min(i + blocco_dim, matrice.shape[1])
        # Cicla il blocco selezionato
        matrice_permutata[:, i:fine_blocco] = np.roll(matrice[:, i:fine_blocco], shift=1, axis=1)
    
    return field(matrice_permutata)


In [53]:
#fino a size 3 provato e funziona
while True:
    id_mat = np.eye(126, dtype=int)

    cols_to_permute = np.random.choice(id_mat.shape[1], size=3, replace=False)
    permuted_cols = np.random.permutation(cols_to_permute)
    matrice_bin1 = id_mat.copy()
    matrice_bin1[:, cols_to_permute] = id_mat[:, permuted_cols]
    matrice_bin1 = field(matrice_bin1)

    cols_to_permute = np.random.choice(id_mat.shape[1], size=3, replace=False)
    permuted_cols = np.random.permutation(cols_to_permute)
    matrice_bin2 = id_mat.copy()
    matrice_bin2[:, cols_to_permute] = id_mat[:, permuted_cols]
    matrice_bin2 = field(matrice_bin2)

    #print(matrice_bin1)
    #print(matrice_bin2)
    A_primo = matrice_bin1 @ A @ matrice_bin2
    A_primo = field(A_primo)
    #print(A_primo)"""
    somme_rig = [field(0)] * A_primo.shape[0]

    #lista per memorizzare i primi elementi della colonna
    primi_colonne = [0] * A_primo.shape[1]
    #lista per memorizzare i secondi elemnti della colonna
    secondi_colonne = [0] * A_primo.shape[1]
    #lista per memorizzare i terzi elementi della colonna
    terzi_colonne = [0] * A_primo.shape[1]
    #lista per memorizzare i quarti elemnti della colonna
    quarti_colonne = [0] * A_primo.shape[1]

    #QUI SI POTREBBE RAGGIONARE SU QUANTI ELEMENTI SALVARE DI OGNI COLONNA POICHÈ LA POSSIBILITÀ CHE QUESTI SIANO UGUALI 
    #PER PIÙ DI TOT RIGHE DIMINUISCE (1/PROB_CONFLITTO_RIGA1 * 1/PROB_CONFLITTO_RIGA2 ....) È DA VEDERE COME VENGONO CREATE 

    colonne_finali = []
    esegui_somma_righe_parallelo(A_primo)
    #ordino in maniera crescente per valore gli indici
    indici_ordinati = np.argsort(somme_rig)
        #print(indici_ordinati)

        #applico gli swap
    A_canonica_righe = A_primo[indici_ordinati, :]
    #ora si passa alle colonne
    esegui_estrazione_seriale(A_canonica_righe)

    #una volta fatta la funzione di estrazione dei valori ordino gli indici in maniera crescente per valore
    #print("Primi colonne: ", primi_colonne)
    indici_ordinati_col = np.argsort(primi_colonne)
    #print("Indici ordinati col: ", indici_ordinati_col)


    for idx in indici_ordinati_col:
        #print("Index: ", idx)
        #qui tengo il primo e il secondo della colonna e gli associo l'indice di riga
        colonne_finali.append((primi_colonne[idx], secondi_colonne[idx], terzi_colonne[idx], quarti_colonne[idx], idx))
        #print("Colonne finali: ", colonne_finali)

    #vado a fare ordinamento per il primo elemento e in caso per il secondo, qui nel caso devo inserire il terzo quarto...
    colonne_finali.sort(key=lambda x: (x[0], x[1], x[2], x[3]))

    #qui prendiamo tutte le righe (prima dei :) e alle colonne per ogni colonna di colonne_finali accediamo al 3 elemento (l'indice)
    A_CF = A_canonica_righe[:, [col[4] for col in colonne_finali]]

    #print(A_CF)

    #devo fare lo stesso per la matrice A per dimostrare che sono CF

    colonne_finali = []

    #creo un thread per ogni colonna
    esegui_somma_righe_parallelo(A)

    #print ("\nSomma degli elementi di ciascuna riga: ", somme_rig)

    #ordino in maniera crescente i valori
    indici_ordinati = np.argsort(somme_rig)
    #print(indici_ordinati)

    #applico gli swap
    A_canonica_righe = A[indici_ordinati, :]

    #print(A_canonica_righe)

    #ora si passa alle colonne
    esegui_estrazione_seriale(A_canonica_righe)

    #una volta fatta la funzione di estrazione dei valori ordino gli indici in maniera crescente per valore
    #print("Primi colonne: ", primi_colonne)
    indici_ordinati_col = np.argsort(primi_colonne)
    #print("Indici ordinati col: ", indici_ordinati_col)


    for idx in indici_ordinati_col:
        #print("Index: ", idx)
        #qui tengo il primo e il secondo della colonna e gli associo l'indice di riga
        colonne_finali.append((primi_colonne[idx], secondi_colonne[idx], terzi_colonne[idx], quarti_colonne[idx], idx))
        #print("Colonne finali: ", colonne_finali)

    #vado a fare ordinamento per il primo elemento e in caso per il secondo, qui nel caso devo inserire il terzo quarto...
    colonne_finali.sort(key=lambda x: (x[0], x[1], x[2], x[3]))

    A_CF1 = A_canonica_righe[:, [col[4] for col in colonne_finali]]
    #print(A_CF1)

    zero = np.subtract(A_CF1,A_CF)
    zero = np.count_nonzero(zero)
    print("Le matrici differiscono per ", zero, " elementi su 15876 (126*126)")
    if zero == 0:
        break


Le matrici differiscono per  619  elementi su 15876 (126*126)
Le matrici differiscono per  740  elementi su 15876 (126*126)
Le matrici differiscono per  618  elementi su 15876 (126*126)
Le matrici differiscono per  498  elementi su 15876 (126*126)


Le matrici differiscono per  617  elementi su 15876 (126*126)
Le matrici differiscono per  740  elementi su 15876 (126*126)
Le matrici differiscono per  620  elementi su 15876 (126*126)
Le matrici differiscono per  250  elementi su 15876 (126*126)
Le matrici differiscono per  496  elementi su 15876 (126*126)
Le matrici differiscono per  500  elementi su 15876 (126*126)
Le matrici differiscono per  614  elementi su 15876 (126*126)
Le matrici differiscono per  250  elementi su 15876 (126*126)
Le matrici differiscono per  496  elementi su 15876 (126*126)
Le matrici differiscono per  621  elementi su 15876 (126*126)
Le matrici differiscono per  494  elementi su 15876 (126*126)
Le matrici differiscono per  623  elementi su 15876 (126*126)
Le matrici differiscono per  743  elementi su 15876 (126*126)
Le matrici differiscono per  623  elementi su 15876 (126*126)
Le matrici differiscono per  374  elementi su 15876 (126*126)
Le matrici differiscono per  620  elementi su 15876 (126*126)
Le matri

In [54]:
print("Questa è A:", A)
print("\nQuesta è A': ", A_primo)
print("\nQuesta è la forma canonica: ", A_CF)

Questa è A: [[ 35  14  87  40  98   3  67  92  81  74   0 104  47  96  15  57  44  19
   19  90  37  89 124  41  59  65   0  76  77  97  96  97 110  56  37  80
   75 125 117 116  97  89  43 113  53  60  32  10  48  91  63  72  43  99
  109  60 112  80 107   9  83 100  88 123  31  29 107  43  46  93 109  16
   45  80  66  38  58  10  82  25  74  11 102 112  86  89  59  24 123  76
    2  51 108  48   5   9  96  29  59  39 101  96  17  63   3  13  12  73
   15  16   4  69  94  52 102  22 108  18  15 106 107 120 116  66 110  55]
 [ 99  56  44  15  17  54  28   7   3  26  71  35  65   1  88  49 114  61
  103 112 112  81 106 117 122  36  62  97  33  38 108 103  60 114  31 118
   99  20 110  73 124  58  62  37 112  26  60  18  33  74  62  56  78  83
   90  62  20  72 122  11  40  29  83  43  20 109  43 103  77  12  73  95
   65  73  28  94  13   9  88  85  96  69  76  81  96 109  25  46  55  24
  115  15 120  48  81 109  32  27  93  57  50   9   9  55  14  48  82  80
   87  25  45  94   4  30

In [55]:
import time

In [56]:
#Voglio vedere il tempo di esecuzione avendo già le matrici

#lista per memorizzare i primi elementi della colonna
primi_colonne = [0] * A_primo.shape[1]
#lista per memorizzare i secondi elemnti della colonna
secondi_colonne = [0] * A_primo.shape[1]
#lista per memorizzare i terzi elementi della colonna
terzi_colonne = [0] * A_primo.shape[1]
#lista per memorizzare i quarti elemnti della colonna
quarti_colonne = [0] * A_primo.shape[1]

#QUI SI POTREBBE RAGGIONARE SU QUANTI ELEMENTI SALVARE DI OGNI COLONNA POICHÈ LA POSSIBILITÀ CHE QUESTI SIANO UGUALI 
#PER PIÙ DI TOT RIGHE DIMINUISCE (1/PROB_CONFLITTO_RIGA1 * 1/PROB_CONFLITTO_RIGA2 ....) È DA VEDERE COME VENGONO CREATE 

colonne_finali = []
start_time = time.perf_counter()

esegui_somma_righe_parallelo(A_primo)
#ordino in maniera crescente per valore gli indici
indici_ordinati = np.argsort(somme_rig)
    #print(indici_ordinati)

    #applico gli swap
A_canonica_righe = A_primo[indici_ordinati, :]
#ora si passa alle colonne
esegui_estrazione_seriale(A_canonica_righe)

#una volta fatta la funzione di estrazione dei valori ordino gli indici in maniera crescente per valore
#print("Primi colonne: ", primi_colonne)
indici_ordinati_col = np.argsort(primi_colonne)
#print("Indici ordinati col: ", indici_ordinati_col)


for idx in indici_ordinati_col:
    #print("Index: ", idx)
    #qui tengo il primo e il secondo della colonna e gli associo l'indice di riga
    colonne_finali.append((primi_colonne[idx], secondi_colonne[idx], terzi_colonne[idx], quarti_colonne[idx], idx))
    #print("Colonne finali: ", colonne_finali)

#vado a fare ordinamento per il primo elemento e in caso per il secondo, qui nel caso devo inserire il terzo quarto...
colonne_finali.sort(key=lambda x: (x[0], x[1], x[2], x[3]))

#qui prendiamo tutte le righe (prima dei :) e alle colonne per ogni colonna di colonne_finali accediamo al 3 elemento (l'indice)
A_CF = A_canonica_righe[:, [col[4] for col in colonne_finali]]

#print(A_CF)

#devo fare lo stesso per la matrice A per dimostrare che sono CF

colonne_finali = []

#creo un thread per ogni colonna
esegui_somma_righe_parallelo(A)

#print ("\nSomma degli elementi di ciascuna riga: ", somme_rig)

#ordino in maniera crescente i valori
indici_ordinati = np.argsort(somme_rig)
#print(indici_ordinati)

#applico gli swap
A_canonica_righe = A[indici_ordinati, :]

#print(A_canonica_righe)

#ora si passa alle colonne
esegui_estrazione_seriale(A_canonica_righe)

#una volta fatta la funzione di estrazione dei valori ordino gli indici in maniera crescente per valore
#print("Primi colonne: ", primi_colonne)
indici_ordinati_col = np.argsort(primi_colonne)
#print("Indici ordinati col: ", indici_ordinati_col)

for idx in indici_ordinati_col:
    #print("Index: ", idx)
    #qui tengo il primo e il secondo della colonna e gli associo l'indice di riga
    colonne_finali.append((primi_colonne[idx], secondi_colonne[idx], terzi_colonne[idx], quarti_colonne[idx], idx))
    #print("Colonne finali: ", colonne_finali)

#vado a fare ordinamento per il primo elemento e in caso per il secondo, qui nel caso devo inserire il terzo quarto...
colonne_finali.sort(key=lambda x: (x[0], x[1], x[2], x[3]))

A_CF1 = A_canonica_righe[:, [col[4] for col in colonne_finali]]
#print(A_CF1)
end_timer = time.perf_counter()
durata = (end_timer - start_time)*1000
print("Durata esecuzione: ", durata, " ms")

zero = np.subtract(A_CF1,A_CF)
zero = np.count_nonzero(zero)
print("Le matrici differiscono per ", zero, " elementi su 15876 (126*126)")

Durata esecuzione:  290.95778099963354  ms
Le matrici differiscono per  0  elementi su 15876 (126*126)
